In [1]:
###### %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

# convolutional autoencoder in keras

import os
#os.environ["KERAS_BACKEND"] = "tensorflow"

from __future__ import print_function
from keras.models import Model
from keras.layers import *
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import backend as K

import matplotlib.pyplot as plt
# %matplotlib inline

from keras.datasets import mnist 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

from PIL import Image

warnings.filterwarnings('ignore')

# utility function for showing images
def show_imgs(x_test, decoded_imgs, n=4):
    plt.figure(figsize=(20, 10))
    for i in range(n):
        ax = plt.subplot(2, n, i+1)
        plt.imshow(x_test[i].reshape(size,size,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if decoded_imgs is not None:
            ax = plt.subplot(2, n, i+ 1 +n)
            plt.imshow(decoded_imgs[i].reshape(size,size,3))
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    plt.show()
    
def show_imgs_gray_out(x_test, decoded_imgs, n=4):
    plt.figure(figsize=(20, 10))
    for i in range(n):
        ax = plt.subplot(2, n, i+1)
        plt.imshow(x_test[i].reshape(size,size,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if decoded_imgs is not None:
            ax = plt.subplot(2, n, i+ 1 +n)
            plt.imshow(decoded_imgs[i].reshape(size,size))
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    plt.show()
    
def show_hidden(x_test, encoded_imgs, n=10):
    
    en0=np.zeros((encoded_imgs.shape[0],encoded_imgs.shape[1],encoded_imgs.shape[2]))
    en1=np.zeros((encoded_imgs.shape[0],encoded_imgs.shape[1],encoded_imgs.shape[2]))
    en2=np.zeros((encoded_imgs.shape[0],encoded_imgs.shape[1],encoded_imgs.shape[2]))
    en3=np.zeros((encoded_imgs.shape[0],encoded_imgs.shape[1],encoded_imgs.shape[2]))

    for i in range(encoded_imgs.shape[0]):
        for j in range(encoded_imgs.shape[1]):
            for k in range(encoded_imgs.shape[2]):
                en0[i][j][k]=encoded_imgs[i][j][k][0]
                en1[i][j][k]=encoded_imgs[i][j][k][1]
                en2[i][j][k]=encoded_imgs[i][j][k][2]
                en3[i][j][k]=encoded_imgs[i][j][k][3]
    
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(5, n, i+1)
        plt.imshow(x_test[i].reshape(size,size,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if decoded_imgs is not None:
            ax = plt.subplot(5, n, i+ 1 +n)
            plt.imshow(en0[i].reshape(int(size/4),int(size/4)))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            
            ax = plt.subplot(5, n, i+ 1 +2*n)
            plt.imshow(en1[i].reshape(int(size/4),int(size/4)))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            
            ax = plt.subplot(5, n, i+ 1 +3*n)
            plt.imshow(en2[i].reshape(int(size/4),int(size/4)))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            
            ax = plt.subplot(5, n, i+ 1 +4*n)
            plt.imshow(en3[i].reshape(int(size/4),int(size/4)))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    plt.show()

def getMaxGrads(input_set, model, layer_name, nof):
    
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    
    out = np.zeros((len(input_set),len(input_set[0]),len(input_set[0][0])))

    for i in range(nof):
        filter_index = i  # can be any integer from 0 to 511, as there are 512 filters in that layer

        layer_output = layer_dict[layer_name].output
        loss = K.mean(layer_output[:, :, :, filter_index])

        # compute the gradient of the input picture wrt this loss
        grads = K.gradients(loss, input_img)[0]

        #normalization trick: we normalize the gradient
        grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)  ### normalize? ... later

        # this function returns the loss and grads given the input picture
        iterate = K.function([input_img], [loss, grads])

        it = iterate([input_set])
        
        len1=len(out)
        len2=len(out[0])
        len3=len(out[0][0])
        
        for j in range(len1):
            for k in range(len2):
                for l in range(len3):
                    if np.mean(abs(it[1][j][k][l])) > out[j][k][l]:   #abs()?
                        out[j][k][l]=np.mean(abs(it[1][j][k][l]))
                        
        return out
    
def lk_relu(x):
    return K.relu(x, alpha=0.3)

print("Starting")

/home/hoffmann/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Starting


In [2]:
### Create input

size=256

train_path = "/home/hoffmann/MA/OIRDS_v1_0/train/train/"
test_path = "/home/hoffmann/MA/OIRDS_v1_0/test/test/"

train_batches = []
listing = os.listdir(train_path) 
for file in listing:
    im = Image.open(train_path + file)   
    im_rsz = im.resize((size,size))
    train_batches.append(np.asarray(im_rsz, dtype="float64") / 255)
train_batches = np.asarray(train_batches)
 
    
print("Train Data done")

test_batches = []
listing = os.listdir(test_path) 

for file in listing:
    im = Image.open(test_path + file)   
    im_rsz = im.resize((size,size))
    test_batches.append(np.asarray(im_rsz, dtype="float64") / 255)
test_batches = np.asarray(test_batches)
    
print("Test Data done")

Train Data done
Test Data done


In [3]:
maxfil = 32
bs_z=16

### pretrain1
    
input_img = Input(shape=(size, size,3))

train1_norm1=BatchNormalization(axis=-1, name='t1n1')(input_img)
train1_c1 = Conv2D(maxfil, (3, 3), activation=lk_relu, padding='same', name='t1c1')(train1_norm1)
train1_norm2=BatchNormalization(axis=-1, name='t1n2')(train1_c1)
train1_decoded = Conv2DTranspose(3, (3, 3), activation=lk_relu, padding='same', name='t1tc1')(train1_norm2)

train1_autoencoder = Model(input_img, train1_decoded)
train1_autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

train1_autoencoder.fit(train_batches, train_batches, epochs=1, validation_data=(test_batches, test_batches), batch_size=bs_z, verbose=1)

### pretrain2

creator_train2 = Model(input_img, train1_norm2)
creator_train2.get_layer("t1n1").set_weights(train1_autoencoder.get_layer('t1n1').get_weights())
creator_train2.get_layer("t1c1").set_weights(train1_autoencoder.get_layer('t1c1').get_weights())
creator_train2.get_layer("t1n2").set_weights(train1_autoencoder.get_layer('t1n2').get_weights())

input_train2 = creator_train2.predict(train_batches)

print(input_train2.shape)

#---

input_img_train2 = Input(shape=(size, size, maxfil))
train2_p1 = MaxPooling2D(pool_size=(2, 2), name='t2p1')(input_img_train2)
train2_c1 = Conv2D(maxfil, (3, 3), activation=lk_relu, padding='same', name='t2c1')(train2_p1)
train2_norm1=BatchNormalization(axis=-1, name='t2n1')(train2_c1)
train2_tc1 = Conv2DTranspose(maxfil, (3, 3), activation=lk_relu, padding='same', name='t2tc1')(train2_norm1)
train2_norm2=BatchNormalization(axis=-1, name='t2n2')(train2_tc1)
train2_decoded = UpSampling2D(size=(2, 2), name='t3u1')(train2_norm2)

train2_autoencoder = Model(input_img_train2, train2_decoded)
train2_autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

train2_autoencoder.fit(input_train2, input_train2, epochs=1, batch_size=bs_z, verbose=1)

### pretrain3

creator_train3 = Model(input_img_train2, train2_norm1)
creator_train3.get_layer("t2c1").set_weights(train2_autoencoder.get_layer('t2c1').get_weights())
creator_train3.get_layer("t2n1").set_weights(train2_autoencoder.get_layer('t2n1').get_weights())

input_train3 = creator_train3.predict(input_train2)

print(input_train3.shape)

#---

input_img_train3 = Input(shape=(size/2, size/2, maxfil))
train3_p1 = MaxPooling2D(pool_size=(2, 2), name='t3p1')(input_img_train3)
train3_c1 = Conv2D(int(maxfil/2), (3, 3), activation=lk_relu, padding='same', name='t3c1')(train3_p1)
train3_norm1=BatchNormalization(axis=-1, name='t3n1')(train3_c1)
train3_tc1 = Conv2DTranspose(maxfil, (3, 3), activation=lk_relu, padding='same', name='t3tc1')(train3_norm1)
train3_norm2=BatchNormalization(axis=-1, name='t3n2')(train3_tc1)
train3_decoded = UpSampling2D(size=(2, 2), name='t3u1')(train3_norm2)

train3_autoencoder = Model(input_img_train3, train3_decoded)
train3_autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

train3_autoencoder.fit(input_train3, input_train3, epochs=1, batch_size=bs_z, verbose=1)

#---

### pretrain3

creator_train4 = Model(input_img_train3, train3_norm1)
creator_train4.get_layer("t3c1").set_weights(train3_autoencoder.get_layer('t3c1').get_weights())
creator_train4.get_layer("t3n1").set_weights(train3_autoencoder.get_layer('t3n1').get_weights())

input_train4 = creator_train4.predict(input_train3)

print(input_train4.shape)

input_img_train4 = Input(shape=(size/4, size/4, int(maxfil/2)))
train4_p1 = MaxPooling2D(pool_size=(2, 2), name='t4p1')(input_img_train4)
train4_c1 = Conv2D(int(maxfil/4), (3, 3), activation=lk_relu, padding='same', name='t4c1')(train4_p1)
train4_norm1=BatchNormalization(axis=-1, name='t4n1')(train4_c1)
train4_tc1 = Conv2DTranspose(int(maxfil/2), (3, 3), activation=lk_relu, padding='same', name='t4tc1')(train4_norm1)
train4_norm2=BatchNormalization(axis=-1, name='t4n2')(train4_tc1)
train4_decoded = UpSampling2D(size=(2, 2), name='t4u1')(train4_norm2)

train4_autoencoder = Model(input_img_train4, train4_decoded)
train4_autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

train4_autoencoder.fit(input_train4, input_train4, epochs=1, batch_size=bs_z, verbose=1)


Train on 845 samples, validate on 37 samples
Epoch 1/1
845/845 [==============================] - 115s 136ms/step - loss: 0.1811 - val_loss: 0.0615
(845, 256, 256, 32)
Epoch 1/1
845/845 [==============================] - 149s 176ms/step - loss: 0.8732
(845, 128, 128, 32)
Epoch 1/1
845/845 [==============================] - 33s 39ms/step - loss: 0.8464
(845, 64, 64, 16)
Epoch 1/1
845/845 [==============================] - 4s 4ms/step - loss: 0.8020


In [4]:
### unroll

input_img = Input(shape=(size, size,3))

n0 = BatchNormalization(axis=-1, name='n0', weights=train1_autoencoder.get_layer("t1n1").get_weights())(input_img)
c1 = Conv2D(maxfil, (3, 3), activation=lk_relu, padding='same', name='c1', weights=train1_autoencoder.get_layer("t1c1").get_weights())(n0)
n1 = BatchNormalization(axis=-1, name='n1', weights=train1_autoencoder.get_layer("t1n2").get_weights())(c1)
p1 = MaxPooling2D(pool_size=(2, 2), name='p1')(n1)
c2 = Conv2D(maxfil, (3, 3), activation=lk_relu, padding='same', name='c2', weights=train2_autoencoder.get_layer("t2c1").get_weights())(p1)
n2 = BatchNormalization(axis=-1, name='n2', weights=train2_autoencoder.get_layer("t2n1").get_weights())(c2)
p2 = MaxPooling2D(pool_size=(2, 2), name='p2')(n2)
c3 = Conv2D(int(maxfil/2), (3, 3), activation=lk_relu, padding='same', name='c3', weights=train3_autoencoder.get_layer("t3c1").get_weights())(p2)
n3 = BatchNormalization(axis=-1, name='n3', weights=train3_autoencoder.get_layer("t3n1").get_weights())(c3)
p3 = MaxPooling2D(pool_size=(2, 2), name='p3')(n3)
c4 = Conv2D(int(maxfil/4), (3, 3), activation=lk_relu, padding='same', name='c4', weights=train4_autoencoder.get_layer("t4c1").get_weights())(p3)
encoded = BatchNormalization(axis=-1, name='n4', weights=train4_autoencoder.get_layer("t4n1").get_weights())(c4)
tc4 = Conv2DTranspose(int(maxfil/2), (3, 3), activation=lk_relu, padding='same', name='tc4', weights=train4_autoencoder.get_layer("t4tc1").get_weights())(encoded)
n4 = BatchNormalization(axis=-1, name='n4', weights=train4_autoencoder.get_layer("t4n2").get_weights())(tc4)
u3 = UpSampling2D(size=(2, 2), name='u3')(n4)
tc3 = Conv2DTranspose(maxfil, (3, 3), activation=lk_relu, padding='same', name='tc3', weights=train3_autoencoder.get_layer("t3tc1").get_weights())(n3)
n5 = BatchNormalization(axis=-1, name='n5', weights=train3_autoencoder.get_layer("t3n2").get_weights())(tc3)
u2 = UpSampling2D(size=(2, 2), name='u2')(n5)
tc2 = Conv2DTranspose(maxfil, (3, 3), activation=lk_relu, padding='same', name='tc2', weights=train2_autoencoder.get_layer("t2tc1").get_weights())(u2)
n6 = BatchNormalization(axis=-1, name='n6', weights=train2_autoencoder.get_layer("t2n2").get_weights())(tc2)
u1 = UpSampling2D(size=(2, 2), name='u1')(n6)
decoded = Conv2DTranspose(3, (3, 3), activation=lk_relu, padding='same', name='tc1', weights=train1_autoencoder.get_layer("t1tc1").get_weights())(u1)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

### finetune

autoencoder.fit(train_batches, train_batches, epochs=1, validation_data=(test_batches, test_batches), batch_size=bs_z, verbose=1)

Train on 845 samples, validate on 37 samples
Epoch 1/1
845/845 [==============================] - 246s 291ms/step - loss: 0.1245 - val_loss: 0.0837


In [5]:
autoencoder.save('CAE_overhead_20ep_2pool_4f.h5')

In [7]:
out = getMaxGrads(test_batches, autoencoder, 'c4', 8)

a=len(test_batches)
b=len(test_batches[0])
c=len(test_batches[0][0])

it_temp = np.zeros((a,b,c))

for i in range(a):
    q = np.percentile(out[i],99)
    for j in range(b):
        for k in range(c):
            if(out[i][j][k]>q):
                it_temp[i][j][k]=out[i][j][k]

KeyError: 'c4'

In [ ]:
writegrads = np.zeros((len(it_temp),len(it_temp[0])*len(it_temp[0])))

for i in range(len(it_temp)):
    temp = []
    for j in it_temp[i]:
        temp.extend(j)
    writegrads[i] = temp

np.savetxt('CAE_overhead_20ep_2pool_4f',writegrads)

In [ ]:
### plot

decoded_imgs = autoencoder.predict(test_batches)
show_imgs(test_batches,decoded_imgs)

print(autoencoder.summary())

autoencoder_show_hidden = Model(input_img, encoded)
encoded_imgs = autoencoder_show_hidden.predict(test_batches)

show_hidden(test_batches, encoded_imgs)

#print grads

show_imgs_gray_out(test_batches,out)

show_imgs_gray_out(test_batches,it_temp)


In [ ]:
print(np.count_nonzero(it_temp[0]==0)/256/256)